In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import pandas as pd
import numpy as np
matplotlib.style.use('ggplot')
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.preprocessing import MinMaxScaler
import math

torch.manual_seed(33)

### Import Data

In [2]:
rppa = pd.read_csv('new_clean_data/consistent_rppa(3omics).csv')
rppa.drop(columns='Unnamed: 0', inplace=True)
rppa

,CELL_LINE_NAME,14-3-3_beta,14-3-3_epsilon_Caution,14-3-3_zeta,4E-BP1,4E-BP1_pS65,4E-BP1_pT37_T46,4E-BP1_pT70,53BP1,A-Raf_pS299_Caution,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,0.108839,0.289584,0.045012,-0.396465,-0.390874,-0.753768,-0.078452,0.407514,-0.079914,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,0.133986,0.024192,0.676852,-0.471224,-0.684351,-0.904702,-0.123291,-3.971552,0.037784,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,0.426952,0.043067,0.223144,0.189454,0.346910,0.398910,0.274279,-0.763783,-0.178467,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,0.098320,0.468521,-0.472370,-0.124265,0.005014,-0.280498,-0.241964,0.662198,1.225230,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,-0.086995,-0.041387,-0.117372,-0.517712,-0.052923,-0.166216,-0.027035,0.267021,-0.275317,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,wsudlcl2,0.089328,0.209751,0.115950,0.217414,-0.196821,-0.417688,0.248472,-0.718635,0.078394,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
521,yapc,0.102917,-0.167794,0.426968,-0.306847,-0.333987,-0.697848,-0.420424,0.145249,0.046873,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
522,yh13,-0.272537,-0.165541,-0.597877,-0.608083,-0.088654,-0.550212,-0.011799,0.236974,-0.441684,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
523,ykg1,-0.292245,0.105827,-0.336982,-0.207507,0.264828,0.272116,0.285140,0.482773,-0.261400,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [3]:
cell_line_cols = ['CELL_LINE_NAME']
drug_resp_cols = ['Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']
rppa_cols = rppa.drop(columns=['CELL_LINE_NAME','Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']).columns

In [4]:
# drug response labels to tensor
drug_resp_labels = np.stack([rppa[col].values for col in drug_resp_cols], 1)
drug_resp_labels = torch.tensor(drug_resp_labels, dtype=torch.float)

In [5]:
# gene_exp values to tensor and scale from 0 to 1
rppa_values = np.stack([rppa[col].values for col in rppa_cols], 1)

In [6]:
scaler = MinMaxScaler()
rppa_values_t = rppa_values.T
scaled_rppa_values_t = scaler.fit_transform(rppa_values_t)
scaled_rppa_values = scaled_rppa_values_t.T
scaled_rppa_values = torch.tensor(scaled_rppa_values, dtype=torch.float)

### Define Dataloaders

In [7]:
batch_size = 32

full_dataset = torch.utils.data.TensorDataset(scaled_rppa_values, drug_resp_labels)
full_loader = DataLoader(full_dataset, shuffle=False, batch_size=batch_size)

### VAE model with Regressor

In [8]:
class VAE_with_regressor(nn.Module):
    def __init__(self, input_size, level_2, level_3, latent_dim):
        super(VAE_with_regressor, self).__init__()
        cat_features = 10
        # Encoder layers
        self.enc_fc1 = nn.Sequential(
                        nn.Linear(input_size, latent_dim),
                        nn.BatchNorm1d(latent_dim),
                        nn.ReLU())

        self.enc_fc2_mean = nn.Sequential(
                    nn.Linear(latent_dim, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        self.enc_fc2_log_var = nn.Sequential(
                    nn.Linear(latent_dim, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        
        # Decoder layers
        self.dec_fc2 = nn.Sequential(
                        nn.Linear(latent_dim, latent_dim),
                        nn.BatchNorm1d(latent_dim),
                        nn.ReLU())
        
        self.dec_fc1 = nn.Sequential(
                    nn.Linear(latent_dim, input_size),
                    nn.BatchNorm1d(input_size),
                    nn.Sigmoid())
        
        # Regression fc layers
        self.r_fc1 = nn.Sequential(
                    nn.Linear(latent_dim, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU())
        self.r_fc2 = nn.Sequential(
                    nn.Linear(64, 32),
                    nn.BatchNorm1d(32),
                    nn.ReLU())
        
        self.r_fc3 = nn.Sequential(
                    nn.Linear(32, 10),
                    nn.BatchNorm1d(10))


    def encode(self, x):
        l2_layer = self.enc_fc1(x)
        mu = self.enc_fc2_mean(l2_layer)
        logvar = self.enc_fc2_log_var(l2_layer)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        l2_layer = self.dec_fc2(z)
        x_hat = self.dec_fc1(l2_layer)
        return x_hat

    def regressor(self, mean):
        level_1_layer = self.r_fc1(mean)
        level_2_layer = self.r_fc2(level_1_layer)
        output_layer = self.r_fc3(level_2_layer)
        return output_layer
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        pred_y = self.regressor(mu)
        return x_hat, mu, logvar, z, pred_y

### Define loss functions

In [9]:
def vae_loss_function(x_hat, x, mean, log_var): # recon loss and kld loss
        bce = torch.nn.functional.binary_cross_entropy(x_hat, x, reduction = 'sum')
        kld = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
        loss = kld + bce
        return loss, kld, bce

In [10]:
def rmse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = torch.sqrt(loss_fn(y_pred, y))
    return loss

In [11]:
def mse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = loss_fn(y_pred, y)
    return loss

In [12]:
def mse_loss_function_mean(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='mean')
    loss = loss_fn(y_pred, y)
    return loss

### Validation functions

In [13]:
def r_squared(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    SS_res = torch.sum(torch.square(y_true - y_pred))
    SS_tot = torch.sum(torch.square(y_true - y_true_mean))
    r_squared = 1 - SS_res / (SS_tot + torch.finfo(torch.float32).eps)
    return r_squared

### Training parameters

In [14]:
epochs = 50

lr = 0.001
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

k = 10
splits = KFold(n_splits=k, shuffle=False)

In [15]:
# Initialize the model
input_size = 214 #dimension of gene expressions
level_2 = 2048
level_3 = 1024
latent_dim = 128 # target latent size
model = VAE_with_regressor(input_size, level_2, level_3, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

### Train Function

In [16]:
def train (model, dataloader, epoch, optimizer, w_recon_loss, w_kl_loss, w_reg_loss):
    model.train()
    train_recon = 0
    train_kl  = 0
    train_reg = 0
    train_total_loss = 0
    batch_num = 0

    for batch_idx, batch in enumerate(dataloader):
        batch_num = batch_num + 1
        # get data values
        gene_exp_values, drug_resp_labels = batch
        gene_exp_values = gene_exp_values.to(device)
        drug_resp_labels = drug_resp_labels.to(device)

        # clear gradients
        optimizer.zero_grad()
        # apply model
        x_hat, mu, logvar, z, pred_y = model(gene_exp_values)

        # losses
        vae_loss, kld, bce = vae_loss_function(x_hat, gene_exp_values, mu, logvar)
        mse_loss = mse_loss_function_mean(pred_y, drug_resp_labels)

        #apply loss weights
        loss = w_recon_loss * bce + w_kl_loss * kld + w_reg_loss * mse_loss

        

        # backward
        loss.backward()

        with torch.no_grad():
            train_recon += bce.item()
            train_kl += kld.item()
            train_reg += mse_loss.item()
            train_total_loss += loss.item()

        # update parameters
        optimizer.step()

    train_recon_ave = train_recon/ len(dataloader.sampler)
    train_kl_ave = train_kl/ len(dataloader.sampler)
    train_reg_ave = train_reg/ batch_num
    train_total_ave = train_total_loss/ len(dataloader.sampler)

    print('=====> Epoch {} \n' 
          'Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'.format(epoch, train_recon_ave, train_kl_ave, train_reg_ave))
    
    return train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave


### Validation function

In [17]:
def test(dataloader, model):
    model.eval()
    test_recon = 0
    test_kl  = 0
    test_reg = 0
    test_rmse = 0
    test_r2 = 0
    batch_num = 0
    with torch.no_grad():
        mean_store = torch.zeros(1, latent_dim).to(device)
        pred_store = torch.zeros(1,10).to(device)
        for batch_idx, batch in enumerate(dataloader):
            batch_num = batch_num + 1
            # get data values
            gene_exp_values, drug_resp_labels = batch
            gene_exp_values = gene_exp_values.to(device)
            drug_resp_labels = drug_resp_labels.to(device)

            # apply model
            x_hat, mu, logvar, z, pred_y = model(gene_exp_values)

            # loss functions
            vae_loss, kld, bce = vae_loss_function(x_hat, gene_exp_values, mu, logvar)
            # bce = mse_loss_function(x_hat, gene_exp_values)
            mse_loss = mse_loss_function(pred_y, drug_resp_labels)
            r2_value = r_squared(drug_resp_labels, pred_y)

            test_recon += bce.item()
            test_kl += kld.item()
            test_reg += mse_loss.item()
            test_r2 += r2_value.item()

            # Get relevant values
            mean_store = torch.cat((mean_store, mu), 0)
            pred_store = torch.cat((pred_store, pred_y), 0)

        # ====== export reduced dimensions and predictions ======
        mean_final = mean_store[1:]
        mean_final_np = mean_final.cpu().numpy()
        mean_final_df = pd.DataFrame(mean_final_np)

        pred_final = pred_store[1:]
        pred_final_np = pred_final.cpu().numpy()
        pred_final_df = pd.DataFrame(pred_final_np)

        input_path = 'C:/Users/mdzak/Desktop/GitHub/FYP_Zaki/notebooks_new/output/'

        mean_path =  input_path + str(latent_dim) + 'D_gene_exp_latent_code.tsv'
        pred_path = input_path + str(latent_dim) + 'D_gene_exp_predictions.tsv'

        mean_final_df.to_csv(mean_path, sep='\t')
        pred_final_df.to_csv(pred_path, sep='\t')

        # print loss
        test_recon_ave = test_recon/ len(dataloader.sampler)
        test_kl_ave = test_kl/ len(dataloader.sampler)
        test_reg_ave = test_reg/ (len(dataloader.sampler) * 10)
        test_rmse_ave = math.sqrt(test_reg_ave)
        test_r2_ave = test_r2/ batch_num

        print('Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'
          'Average RMSE Loss: {:.3f} \n'
          'Average R2: {:.3f} \n'.format(test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave))
        
        return test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave

### Execute Training (Cross Validation)

In [18]:
history_train = {'Train Recon Loss': [], 'Train KL Loss': [], 'Train MSE Loss': [], 'Train Total Loss': []}
history_test = {'Test Recon Loss': [], 'Test KL Loss': [], 'Test MSE Loss': [], 'Test RMSE Loss': [], 'Test R2': []}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(full_dataset)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=test_sampler)

    print("Start Training (Unsupervised Phase)")
    for epoch in range(1, epochs + 1):
       train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=1, w_kl_loss=1, w_reg_loss=0)
       history_train['Train Recon Loss'].append(train_recon_ave)
       history_train['Train KL Loss'].append(train_kl_ave)
       history_train['Train MSE Loss'].append(train_reg_ave)
       history_train['Train Total Loss'].append(train_total_ave)

    print("Start Training (Supervised Phase)")
    for epoch in range(1, epochs + 1):
        train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=0, w_kl_loss=0, w_reg_loss=1)
        history_train['Train Recon Loss'].append(train_recon_ave)
        history_train['Train KL Loss'].append(train_kl_ave)
        history_train['Train MSE Loss'].append(train_reg_ave)
        history_train['Train Total Loss'].append(train_total_ave)

    test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave = test(test_loader, model)
    history_test['Test Recon Loss'].append(test_recon_ave)
    history_test['Test KL Loss'].append(test_kl_ave)
    history_test['Test MSE Loss'].append(test_reg_ave)
    history_test['Test RMSE Loss'].append(test_rmse_ave)
    history_test['Test R2'].append(test_r2_ave)

Fold 1
Start Training (Unsupervised Phase)
=====> Epoch 1 
Average Recon Loss: 164.746 
Average KL Loss: 100.898 
Average MSE Loss: 13.474 

=====> Epoch 2 
Average Recon Loss: 153.640 
Average KL Loss: 90.564 
Average MSE Loss: 13.569 

=====> Epoch 3 
Average Recon Loss: 151.433 
Average KL Loss: 84.129 
Average MSE Loss: 13.595 

=====> Epoch 4 
Average Recon Loss: 150.422 
Average KL Loss: 79.816 
Average MSE Loss: 13.521 

=====> Epoch 5 
Average Recon Loss: 149.545 
Average KL Loss: 76.048 
Average MSE Loss: 13.492 

=====> Epoch 6 
Average Recon Loss: 149.106 
Average KL Loss: 73.610 
Average MSE Loss: 13.549 

=====> Epoch 7 
Average Recon Loss: 148.379 
Average KL Loss: 70.931 
Average MSE Loss: 13.563 

=====> Epoch 8 
Average Recon Loss: 147.545 
Average KL Loss: 68.686 
Average MSE Loss: 13.543 

=====> Epoch 9 
Average Recon Loss: 147.086 
Average KL Loss: 66.497 
Average MSE Loss: 13.540 

=====> Epoch 10 
Average Recon Loss: 146.520 
Average KL Loss: 64.084 
Average MSE 

### Performance of CV

In [19]:
train_avg_cv_recon_loss = np.mean(history_train['Train Recon Loss'])
train_avg_cv_kl_loss = np.mean(history_train['Train KL Loss'])
train_avg_cv_mse_loss = np.mean(history_train['Train MSE Loss'])
train_avg_cv_total_loss = np.mean(history_train['Train Total Loss'])

test_avg_recon_loss =  np.mean(history_test['Test Recon Loss'])
test_avg_kl_loss =  np.mean(history_test['Test KL Loss'])
test_avg_mse_loss =  np.mean(history_test['Test MSE Loss'])
test_avg_rmse_loss =  np.mean(history_test['Test RMSE Loss'])
test_avg_r2_loss =  np.mean(history_test['Test R2'])

print('Performance of {} fold cross validation {} dimension:'.format(k, latent_dim))
print("\nAverage Training Recon Loss: {:.4f} \n Average Training KL Loss: {:.4f} \n Average Training RMSE Loss: {:.3f} \n Average Training Total Loss: {:.3f} \n".format(train_avg_cv_recon_loss, train_avg_cv_kl_loss, train_avg_cv_mse_loss, train_avg_cv_total_loss))
print("\nAverage Testing Recon Loss: {:.4f} \n Average Testing KL Loss: {:.4f} \n Average Testing MSE Loss: {:.3f} \n Average Testing RMSE Loss: {:.3f} \n Average Testing R2 Loss: {:.3f} \n".format(test_avg_recon_loss, test_avg_kl_loss, test_avg_mse_loss, test_avg_rmse_loss, test_avg_r2_loss))  


Performance of 10 fold cross validation 128 dimension:

Average Training Recon Loss: 141.8852 
 Average Training KL Loss: 4.1776 
 Average Training RMSE Loss: 3.760 
 Average Training Total Loss: 73.926 


Average Testing Recon Loss: 141.9248 
 Average Testing KL Loss: 2.4467 
 Average Testing MSE Loss: 3.080 
 Average Testing RMSE Loss: 1.689 
 Average Testing R2 Loss: 0.650 



In [20]:
np.mean(history_test['Test RMSE Loss'])

1.6886974244672803

In [21]:
history_test['Test RMSE Loss']

[2.8408045324750066,
 2.145750034399806,
 1.798975488613061,
 1.6798746604810424,
 1.629911991979736,
 1.5059213210906819,
 1.615873705703033,
 1.3863239636441327,
 1.17553140514095,
 1.1080071411453545]

In [22]:
history_test['Test MSE Loss']

[8.070170391730542,
 4.604243210126769,
 3.2363128086306014,
 2.8219788749262973,
 2.6566131015993513,
 2.267799025315505,
 2.611047832782452,
 1.9218941321739784,
 1.3818740844726562,
 1.2276798248291017]

### Save model

In [23]:
# torch.save(model.state_dict(), 'trained_models/VAE_gene_exp' + str(latent_dim) + '.pt')